In [1]:
%matplotlib widget      

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import pandas as pd
import sys
import os
from scipy import interpolate
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from tkinter import _flatten
from astropy.stats import poisson_conf_interval
from pwkit.bblocks import tt_bblock as bb

In [4]:
srcid=np.linspace(1,1055,1055)
srcid=srcid.astype(int)

In [5]:
path='/Users/baotong/Desktop/CDFS/'

In [6]:
def plot_longT_V(data_file,epoch_file):
    epoch_info = np.loadtxt(epoch_file)
    t_start = epoch_info[:, 0]
    t_end = epoch_info[:, 1]
    obsID = epoch_info[:, 2]
    expT = epoch_info[:, 3]
    time = np.loadtxt(data_file)
    cts=[]
    for i in range(len(obsID)):
        cts.append(len(np.where(time[:,2]==obsID[i])[0]))
    cts=np.array(cts)
    CR=cts/expT
#     print(obsID[np.where(CR>0.0006)])
    if np.min(CR)!=0:
        VI=np.max(CR)/np.min(CR)
    else:VI=0
    plt.title(dataname[0:-4]+', VI={0}'.format(VI))
    plt.scatter(t_start,CR,marker='+')
    plt.show()
    

In [16]:
path='/Users/baotong/Desktop/CDFS/txt_all_obs_0.5_8/'
id=780
dataname='{0}.txt'.format(id)
data_file=path+dataname
plot_longT_V(path + dataname, path + 'epoch_src_' + dataname)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def get_bright_id(cts_limit):
    cts_num=[]
    for i in range(len(srcid)):
        dataname='{0}.txt'.format(srcid[i])
        data_file=path+dataname
        time=np.loadtxt(data_file)[:,0]
        cts_num.append(len(time))
    cts_num=np.array(cts_num)
    return srcid[np.where(cts_num>cts_limit)]

In [10]:
# get_bright_id(1000)

In [8]:
bright_id=get_bright_id(1500)
for id in bright_id:
    dataname='{0}.txt'.format(id)
    data_file=path+dataname
    time=np.loadtxt(data_file)[:,0]
    plot_longT_V(path + dataname, path + 'epoch_src_' + dataname)

NameError: name 'get_bright_id' is not defined

In [11]:
obs_ID=[581,1431,441,582,2406,2405,2312,1672,2409,2313,2239,8591,9593,9718,8593,8597,8595,8592,8596,
9575,9578,8594,9596,12043,12123,12044,12128,12045,12129,12135,12046,12047,12137,12138,12055,12213,12048,
12049,12050,12222,12219,12051,12218,12223,12052,12220,12053,12054,12230,12231,12227,12233,12232,12234,16183,
16180,16456,16641,16457,16644,16463,17417,17416,16454,16176,16175,16178,16177,16620,16462,17535,17542,16184,
16182,16181,17546,16186,16187,16188,16450,16190,16189,17556,16179,17573,17633,17634,16453,16451,16461,16191,
16460,16459,17552,16455,16458,17677,18709,18719,16452,18730,16185]

source_id=np.linspace(1,1055,1055)
source_id=source_id.astype(int)

In [12]:
path='/Users/baotong/Desktop/CDFS/'

In [68]:
# def filter_evt_byobstime(epochtime,src_evt,bkg_evt,epoch):
#     useobs=epoch[np.where((epoch[:,0]>epoch1[0])&(epoch[:,0]<epoch1[1]))]
#     print(src_evt[:,-1])
#     print(useobs)
# epoch1=[55e6,100e6]
# epoch2=[300e6,320e6]
# epoch3=[380e6,400e6]
# epoch4=[500e6,580e6]
# k=0;i=122
# src_evt=np.loadtxt(path+'txt_obs_each/txt_{0}_0.5_8/{1}.txt'.format(obs_ID[k],source_id[i]))
# bkg_evt=np.loadtxt(path+'txt_obs_each/txt_{0}_0.5_8/{1}_bkg.txt'.format(obs_ID[k],source_id[i]))
# epoch=np.loadtxt(path+'txt_all_obs_0.5_8/epoch_src_{0}.txt'.format(source_id[i]))
# filter_evt_byobstime(epoch1,src_evt,bkg_evt,epoch)

## 给定epoch（起止时间），返回该时间内的源和背景的事件和epoch文件

In [13]:
def get_evt_from_epoch(epochnum,source_id):
    src_evt_all=[];bkg_evt_all=[];epoch_all=[]
    for i in range(len(source_id)):
        epoch=np.loadtxt(path+'txt_all_obs_0.5_8/epoch_src_{0}.txt'.format(source_id[i]))
        useobs=epoch[np.where((epoch[:,0]>epochnum[0])&(epoch[:,0]<epochnum[1]))]
#         print(useobs)
        use_obsid=useobs[:,2].astype(int)
        src_evt=[];bkg_evt=[];
        for k in range(len(use_obsid)):
            src_evt_k=np.loadtxt(path+'txt_obs_each/txt_{0}_0.5_8/{1}.txt'.format(use_obsid[k],source_id[i]))
            bkg_evt_k=np.loadtxt(path+'txt_obs_each/txt_{0}_0.5_8/{1}_bkg.txt'.format(use_obsid[k],source_id[i]))
            if len(src_evt_k)==0:
                continue
            elif type(src_evt_k[0])==type(np.array([1.2])[0]):src_evt_k=[src_evt_k]
                
            if len(bkg_evt)==0:
                bkg_evt=bkg_evt_k
               
            if len(bkg_evt_k)==0:
                bkg_evt=bkg_evt
            elif type(bkg_evt_k[0])==type(np.array([1.2])[0]):
                bkg_evt_k=[bkg_evt_k]
                bkg_evt=np.row_stack((bkg_evt,bkg_evt_k))    
            else:
                bkg_evt=np.row_stack((bkg_evt,bkg_evt_k))   
            if len(src_evt)==0:
                src_evt=src_evt_k
            else:src_evt=np.row_stack((src_evt,src_evt_k))
        src_evt_all.append(src_evt);bkg_evt_all.append(bkg_evt);epoch_all.append(useobs)
    return [src_evt_all,bkg_evt_all,epoch_all]

In [14]:
np.row_stack((np.array([[4,40],[5,50]]),[[2,20]]))

array([[ 4, 40],
       [ 5, 50],
       [ 2, 20]])

In [15]:
epoch1=[55e6,100e6]
epoch2=[300e6,320e6]
epoch3=[380e6,400e6]
epoch4=[500e6,580e6]
EPALL=np.array([epoch1,epoch2,epoch3,epoch4])

In [16]:
source_id=srcid
for i in range(len(EPALL)):
    EVTALL=get_evt_from_epoch(EPALL[i],source_id)
    for j in range(len(source_id)):
        if len(EVTALL[0][j])==0:
            np.savetxt(path+'txt_all_obs_0.5_8_ep{0}/{1}.txt'.format(i+1,source_id[j]),
                   EVTALL[0][j])
        else:
            np.savetxt(path+'txt_all_obs_0.5_8_ep{0}/{1}.txt'.format(i+1,source_id[j]),
                   EVTALL[0][j],fmt="%.7f  %8.3f  %10d")
        if len(EVTALL[1][j])==0:    
            np.savetxt(path+'txt_all_obs_0.5_8_ep{0}/{1}_bkg.txt'.format(i+1,source_id[j]),
                   EVTALL[1][j])
        else:
            np.savetxt(path+'txt_all_obs_0.5_8_ep{0}/{1}_bkg.txt'.format(i+1,source_id[j]),
                   EVTALL[1][j],fmt="%.7f  %8.3f  %10d")
        np.savetxt(path+'txt_all_obs_0.5_8_ep{0}/epoch_src_{1}.txt'.format(i+1,source_id[j]),
                   EVTALL[2][j],fmt='%15.2f %15.2f %10d %20.2f')

In [ ]:
def make_aprates_input():
    os.chdir(path+'txt_all_obs_0.5_8_ep{0}'.format(i+1))
    os.system('mkdir aprates')
    src_cts=len(np.loadtxt('{0}.txt'.format(source_id[j])))
    bkg_cts=len(np.loadtxt('{0}_bkg.txt'.format(source_id[j])))
    epoch=np.loadtxt('epoch_src_{0}.txt'.format(source_id[j]))
    exp_s=1
    aprates_text='aprates n={0} m={1} A_s={2} A_b={3} alpha=0.9 beta=0.02 T_s=1 ' \
                             'E_s={4} eng_s=1 flux_s=1 T_b=1 E_b={5} eng_b=1 flux_b=1 clobber=yes ' \
                             'outfile={6} conf=0.9973'.format(src_cts,bkg_cts,backscale,b_backscale,exp_s,exp_s,str(int(ID))+'_'+str(int(obs))+'_out.par')
    with open('aprates/'+'run_'+str(int(ID))+'_'+str(int(obs))+'.e', 'w+') as f:
        f.writelines(aprates_text)
    

In [227]:
source_id=srcid
def write_4ep_info():
    for j in range(len(source_id)):
        src_cts=[];bkg_cts=[];exptime=[];
        for i in range(len(EPALL)):
            os.chdir(path+'txt_all_obs_0.5_8_ep{0}'.format(i+1))
            src_cts.append(len(np.loadtxt('{0}.txt'.format(source_id[j]))))
            bkg_cts.append(len(np.loadtxt('{0}_bkg.txt'.format(source_id[j]))))
            epoch=np.loadtxt('epoch_src_{0}.txt'.format(source_id[j]))
            if len(epoch)==0:
                exptime.append(0)
            elif type(epoch[0])==type(np.array([1.2])[0]):
                exptime.append(epoch[-1])
            else:
                exptime.append(np.sum(epoch[:,-1]))
        info=np.column_stack((src_cts,bkg_cts,exptime,obstime))
        np.savetxt(path+'ep1-4_info/{0}_4ep_info.txt'.format(source_id[j]),info,fmt='%10d  %10d  %10.5f')


In [226]:
write_4ep_info()

In [298]:
  def plot_var_4ep(src_ID):
    ep4info=np.loadtxt(path+'ep1-4_info/{0}_4ep_info.txt'.format(src_ID))
    expT=ep4info[:,-1]
    src_cts=ep4info[:,0];bkg_cts=ep4info[:,1]
    err=poisson_conf_interval(src_cts, background=bkg_cts/12., confidence_level=0.90,
    interval='kraft-burrows-nousek').T  
    x=[np.mean(EPALL[i]) for i in range(len(EPALL))]
    y=(src_cts-bkg_cts/12.)
    yerr=[0,0]
    yerr[0]=y-err[:,0]
    yerr[1]=err[:,1]-y
    
    y/=expT
    yerr/=expT
    plt.errorbar(x, y, yerr = yerr, fmt = '.', capsize = 1, elinewidth = 1, ecolor = 'red')
    plt.semilogy()
    plt.show()

In [299]:
%matplotlib widget  
plot_var_4ep('479')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
def bb_all_obs(source_id):
    epoch=np.loadtxt(path+'txt_all_obs_0.5_8_ep3/epoch_src_{0}.txt'.format(source_id))
    tstart=np.array(epoch[:,0])
    tstop=np.array(epoch[:,1])
    obs_id=epoch[:,2].astype(int)
    temp=np.loadtxt(path+'txt_all_obs_0.5_8_ep3/{0}.txt'.format(source_id))
    if len(temp)==0:src_evt=np.array([])
    elif type(temp[0])==type(np.array([1.2])[0]):src_evt=np.array([temp[0]])
    else: src_evt=temp[:,0]
    return bb(tstart,tstop,src_evt,p0=0.9,intersect_with_bins=True)

In [91]:
# %matplotlib widget 
test_bb=bb_all_obs('106')
print(test_bb)
plt.step(test_bb.midpoints,test_bb.rates)
plt.show()

{counts=[192 192 192 192 192 192 192 192 192 192 192 192 192  28  28  28  28  28
  28  28  94  94  94  94  94  94  94  94  94  94  94  94], finalp0=0.31870793094203875, ledges=[3.85263293e+08 3.85545829e+08 3.85730815e+08 3.86082193e+08
 3.86181208e+08 3.86694947e+08 3.86933446e+08 3.87101510e+08
 3.87465343e+08 3.87794853e+08 3.87981284e+08 3.90330561e+08
 3.90492993e+08 3.90546525e+08 3.90985396e+08 3.91459971e+08
 3.91934506e+08 3.92092944e+08 3.92228686e+08 3.92556304e+08
 3.92638395e+08 3.92777534e+08 3.93004667e+08 3.93252653e+08
 3.94686546e+08 3.95062349e+08 3.95207363e+08 3.95291961e+08
 3.95528498e+08 3.95662985e+08 3.95869910e+08 3.96215805e+08], midpoints=[3.85330027e+08 3.85559349e+08 3.85782578e+08 3.86094416e+08
 3.86232903e+08 3.86735180e+08 3.86965909e+08 3.87141853e+08
 3.87471367e+08 3.87842862e+08 3.88002566e+08 3.90372471e+08
 3.90519759e+08 3.90552082e+08 3.91056297e+08 3.91505403e+08
 3.91950288e+08 3.92109657e+08 3.92246785e+08 3.92586464e+08
 3.92683576e+08 3.9

In [400]:
def bb_single_obs(source_id):
    epoch=np.loadtxt(path+'txt_all_obs_0.5_8/epoch_src_{0}.txt'.format(source_id))
    tstart=np.array(epoch[:,0])
    tstop=np.array(epoch[:,1])
    obs_id=epoch[:,2].astype(int)
    
    BB_all=[]
    for i in range(len(obs_id)):
        temp=np.loadtxt(path+'txt_obs_each/txt_{0}_0.5_8/{1}.txt'.format(obs_id[i],source_id))
        if len(temp)==0:BB_all.append([])
        elif type(temp[0])==type(np.array([1.2])[0]):src_evt=np.array([temp[0]])
        else: src_evt=temp[:,0];BB_all.append(bb([tstart[i]],[tstop[i]],src_evt,p0=0.5,intersect_with_bins=False))
    return BB_all

In [586]:
# %matplotlib widget  
# test_bb=bb_all_obs('855')
# print(test_bb)
# plt.step(test_bb.midpoints,test_bb.rates)
# plt.show()

In [390]:
test_bb=bb_single_obs('479')
block=0;
for i in range(len(test_bb)):
    if type(test_bb[i])==type([]):
        block+=0
    else:
        block+=test_bb[i].nblocks
print(block)

121


In [565]:
a=np.array([10,10,10,2.213,24,2,2,4,4,4,4,5,5])
temp=np.unique(a,return_index='True')
print(temp)
print(a[np.sort(temp[1])])

(array([ 2.   ,  2.213,  4.   ,  5.   , 10.   , 24.   ]), array([ 5,  3,  7, 11,  0,  4]))
[10.     2.213 24.     2.     4.     5.   ]


In [566]:
def cut_bb_res(BB_Holder):
    [counts,index1]=np.unique(BB_Holder.counts,return_index='True')
    [rates,index2]=np.unique(BB_Holder.rates,return_index='True')
    if (index1==index2).all()=='False':
        print('error')
    index1=np.sort(index1)
    counts=BB_Holder.counts[index1];rates=BB_Holder.rates[index1]
    ledges=np.array(np.split(BB_Holder.ledges,index1[1:]))
    redges=np.array(np.split(BB_Holder.redges,index1[1:]))
    midpoints=np.array(np.split(BB_Holder.midpoints,index1[1:]))
    blocks=len(index1)
    return [counts,rates,ledges,redges,midpoints,blocks]

In [630]:
def plot_cutbb_res(source_id):
    %matplotlib widget 
    [counts,rates,ledges,redges,midpoints,blocks]=cut_bb_res(bb_all_obs(source_id))
    expT=counts/rates
    ledges_plt=[];redges_plt=[];
    for i in range(len(counts)):
        ledges_plt.append(ledges[i][0])
        redges_plt.append(redges[i][-1])
    ledges_plt=np.array(ledges_plt);redges_plt=np.array(redges_plt)
    midpoints_plt=(ledges_plt+redges_plt)/2.
    VI=np.max(rates)/np.min(rates)
#     ledges_plt=np.concatenate((ledges_plt,[redges_plt[-1]]))
#     rates=np.concatenate((np.array([0]),rates))
#     plt.step(ledges_plt,rates)
#     plt.show()
    x=midpoints_plt;xerr=[x-ledges_plt,redges_plt-x]
    y=rates
    ylim=poisson_conf_interval(counts,interval='frequentist-confidence',sigma=2)/expT
    yerr=[y-ylim[0],ylim[1]-y]
    plt.errorbar(x, y, xerr=xerr,yerr = yerr, fmt = '.', capsize = 1, elinewidth = 1, ecolor = 'red')
    
    print(EPALL[1])
    plt.fill_between(EPALL[0],np.max(ylim[1]),hatch='\\',color='green')
    plt.fill_between(EPALL[1],np.max(ylim[1]),hatch='\\',color='green')
    plt.fill_between(EPALL[2],np.max(ylim[1]),hatch='\\',color='green')
    plt.fill_between(EPALL[3],np.max(ylim[1]),hatch='\\',color='green')
#     plt.semilogy()
    plt.show()
    

In [633]:
plot_cutbb_res('780')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[3.0e+08 3.2e+08]


In [218]:
from scipy.optimize import curve_fit
import astropy.units as u
import astropy.constants as c

In [219]:
def EddL(mass):
    L=4*np.pi*c.G.cgs*mass*u.Msun.cgs*c.m_p*c.c.cgs/c.sigma_T.cgs
    L=L.to(u.erg/u.s)
    return L.value

In [220]:
def plot_QPO_MASS():
    %matplotlib widget    
    srcid=[127,384,106,124,51,141,867]
    
    qpo=[291.3885776,893.0091637,274.5237018,282.7124869,214.7550065,349.7908837,750.11999]
    srcid=np.array(srcid);qpo=np.array(qpo)
    catalog=fits.open('/Users/baotong/Desktop/CDFS/7Ms_catalog.fit')
    zF=catalog[1].data['zF']
    Seq=catalog[1].data['Seq']
    LX=catalog[1].data['LX']
    zF_use=zF[srcid-1]
    LX_use=LX[srcid-1]
    
#     x=LX_use
    freq_qpo=1./(qpo/(1+zF_use))
    
    QPOMASS=np.loadtxt('/Users/baotong/Desktop/CDFS/QPO_MASS.txt')
    MASS=QPOMASS[:,0];freqQPO=QPOMASS[:,1]
    x2=MASS;y2=freqQPO
    
    def func(x,a,b):
        return a*x+b
    def func1(y,a,b):
        return (y-b)/a
    
    xdata=np.log10(x2)
    np.random.seed(1729)
    y_noise = 0.2 * np.random.normal(size=xdata.size)
    ydata = np.log10(y2) + y_noise
    
    plt.scatter(x2, y2)
    
    popt, pcov = curve_fit(func, xdata, ydata)
    plt.plot(10**xdata, 10**func(xdata, *popt), 'r--',label='fit: a=%5.3f, b=%5.3f' % tuple(popt))

    
    y=freq_qpo
    mass_cdfs=10**(func1(np.log10(freq_qpo),popt[0],popt[1]))
    x=mass_cdfs
    Ledd=EddL(x)
    print(Ledd)
    print(LX_use/Ledd)
    plt.loglog()
    plt.scatter(x,y,marker='^')
#     plt.scatter(x2,y2,marker='^')
    plt.show()
    
    

In [221]:
plot_QPO_MASS()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[6.75184850e+42 1.65941164e+43 7.66413793e+42 6.55275014e+42
 5.17152029e+42 1.41031389e+43 2.16641806e+43]
[3.55458213 2.1091813  8.61153604 0.39677997 5.41426861 0.13472178
 0.30003443]


In [217]:
plosimple_HRS()

NameError: name 'plosimple_HRS' is not defined

In [213]:
def simple_HR(Psrc_id):
    soft_cts=[];hard_cts=[]
    for i in [1,2,3,4]:
        evt_file='/Users/baotong/Desktop/CDFS/txt_all_obs_0.5_8_ep{0}/{1}.txt'.format(i,Psrc_id)
        epoch_file='/Users/baotong/Desktop/CDFS/txt_all_obs_0.5_8_ep{0}/epoch_src_{1}.txt'.format(i,Psrc_id)
        evt=np.loadtxt(evt_file)
        if len(evt)<4:
            print(evt)
            soft_cts.append(0);hard_cts.append(0)
        else:
            time=np.loadtxt(evt_file)[:,0]
            energy=np.loadtxt(evt_file)[:,1]
            soft_cts.append(len(np.where((energy>500)&(energy<2000))[0]))
            hard_cts.append(len(np.where((energy>2000)&(energy<8000))[0]))
        
    soft_cts=np.array(soft_cts)
    hard_cts=np.array(hard_cts)
     
    return [soft_cts,hard_cts]

In [214]:
%matplotlib widget
EP=[3,4,3,4,2,2,3];srcid=[127,384,106,124,51,141,867]
for i in [0]:
# for i in range(len(srcid)):
    useid=srcid[i];EP_I=EP[i]
    HR=simple_HR(str(useid))
    print(HR)
    res=(HR[1]-HR[0])/(HR[1]+HR[0])
    X=[1,2,3,4]
    Y=res
    plt.figure(1)
    plt.scatter(X,Y,marker='^')
    plt.plot([EP_I,EP_I],[0,1],'--',color='red')
    plt.show()

[array([66, 46, 62, 98]), array([35, 45, 59, 90])]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …